<a href="https://colab.research.google.com/github/raminnourizade/Proposal/blob/main/eeg_tourch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mne

In [2]:
import gdown
import os
import numpy as np
import pandas as pd
import mne
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import glob
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
import keras
from tensorflow.keras import layers
import sys
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models, optimizers
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom
from scipy import signal

from tensorflow.keras import regularizers

from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from tensorflow.keras.optimizers import Adam


In [3]:
def load_data():
  path="dataset"
  if not os.path.exists(path):
      os.mkdir(path)
      print(f"Folder {path} created!")

      file_id = "1hG5v_COjPNzejRaL9XJAFERee9i2_V04"  # Replace this with your file's ID
      output_file = path+"/eeg.zip"  # Replace "data_file.ext" with the desired output filename and extension
      gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file)
      !unzip "dataset/eeg.zip" -d "dataset"
      os.remove("dataset/eeg.zip")

  else:
      print(f"Folder {path} already exists")


  ds_dir = '/content/dataset/'
  scaler = StandardScaler()
  all_eeg=[]

  for (i, item) in enumerate(glob.glob(ds_dir + '*.edf')):
    print(item)
    raw = mne.io.read_raw_edf(item)
    # Filter EEG channels ('P3' and 'C3')
    eeg_channels = [ch for ch in raw.info['ch_names'] if ch in channels]
    if len(eeg_channels) != len(channels):
        print(f"Error: channels not found in {item}. Skipping...")
        continue

    # Create a new Raw object with only EEG channels
    raw = raw.copy().pick_channels(eeg_channels)


    # Down Sampling
    raw = raw.copy().resample(sfreq=128)
    # Apply bandpass filter (example: 0.1 Hz - 40 Hz)
    raw.load_data()  # Load the data into memory
    raw.filter(l_freq=0.1, h_freq=40)
    #raw= raw.to_data_frame().to_numpy().astype(np.float32)

   # print("mean of data is  {:.6f}".format(np.mean(np.mean(raw))))
    # Segment data into epochs (e.g., 1-second epochs)
    events = mne.make_fixed_length_events(raw, duration=1.0)
    epochs = mne.Epochs(raw, events, tmin=0, tmax=1.0, baseline=None)

    X = epochs.get_data()
    # Standardize features (Z-score normalization)
    n_samples, n_channels, n_time_points = X.shape

    # Reshape to 2D (n_samples x (n_channels * n_time_points))
    #X_reshaped = X.reshape(n_samples, -1)
    X_reshaped=X
    # Apply StandardScaler

    # Compute mean and standard deviation along the time axis (axis 2)
    mean = np.mean(X_reshaped, axis=(0, 1, 2))
    std = np.std(X_reshaped, axis=(0, 1, 2))
    print("mean of data is  {:.6f}".format(np.mean(mean)))
     # Perform scaling
    eeg_data_scaled = (X_reshaped - mean) / std
    X_scaled = eeg_data_scaled

    all_eeg.append(X_scaled)


  #all_eeg=np.array(all_eeg)

  lb=LabelBinarizer()
  all_labels = pd.read_excel("/content/dataset/states.xlsx",usecols=["status"])
  all_labels=lb.fit_transform(all_labels)


  return all_eeg, all_labels, n_samples, n_channels ,n_time_points
  #return  X_train, X_test, y_train, y_test, n_samples, n_channels ,n_time_points

In [4]:
def apply_augmentation(data, noise_level=0.2):
  np.random.seed(42)
  noise = np.random.normal(scale=noise_level, size=np.shape(data))
  augmented_data=data + noise

  return np.array(augmented_data)

In [5]:
def lr_scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


In [6]:
channels= ['T3', 'T5', 'T4', 'T6','Fp2','F3','Fz','Pz','C3','P3','O1','O2']
all_eeg, all_labels, n_samples, n_channels ,n_time_points = load_data()
#X_train, X_test, y_train, y_test, n_samples, n_channels ,n_time_points = load_data()


Folder dataset already exists
/content/dataset/24.edf
Extracting EDF parameters from /content/dataset/24.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 4225 samples (33.008 s)

Not setting metadata
1811 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1811 events and 12

In [7]:
print( "n_samples= {} and  n_channels = {} and n_time_points= {}".format(n_samples, n_channels ,n_time_points))

n_samples= 1981 and  n_channels = 12 and n_time_points= 129


In [8]:
all_eeg=np.array(all_eeg)

<ipython-input-8-ae1a91f1323d>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_eeg=np.array(all_eeg)


In [9]:
for x in range(len(all_eeg)):
  all_eeg[x]=all_eeg[x][0:1802,0:12,0:129]
  #all_eeg[x]=all_eeg[x][0:1802,0:12,0:257]


In [10]:
all_eeg=np.stack( all_eeg, axis=0 )




In [11]:
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
fold_accuracies = []

#X_train, X_test, y_train, y_test = train_test_split(all_eeg, all_labels, test_size=0.2, random_state=42)

In [12]:
mlp_head_units =[16, 8] #=[32, 16]#[64, 32] [128, 64] #   [256, 128]
input_shape =np.shape(all_eeg[0])#  (1802, 12, 257)

In [13]:
np.shape(all_eeg)

(27, 1802, 12, 129)

In [14]:
lr_callback = LearningRateScheduler(lr_scheduler)


In [15]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [16]:
all_eeg_augmented = apply_augmentation(all_eeg)


In [17]:
class LayerNorm(layers.Layer):
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_first"):
        super(LayerNorm, self).__init__()
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError
        self.normalized_shape = normalized_shape
        self.layer_norm = layers.LayerNormalization(epsilon=self.eps, axis=-1 if self.data_format == "channels_last" else 1)

    def call(self, x):
        return self.layer_norm(x)


In [18]:
class Attention(layers.Layer):
    def __init__(self, embed_dim, num_heads=1):
        super(Attention, self).__init__()
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.to_qkv = layers.Conv2D(2 * embed_dim + num_heads, kernel_size=1, padding='valid')
        self.attend = layers.Softmax(axis=-1)

    def call(self, x):
        qkv = self.to_qkv(x)
        i, k, v = tf.split(qkv, [1, self.head_dim, self.head_dim], axis=-1)
        scores = self.attend(i)
        attn_vector = k * score
        attn_vector = tf.reduce_sum(attn_vector, axis=-2, keepdims=True)
        return attn_vector

In [19]:
class FeedForward(layers.Layer):
    def __init__(self, dim):
        super(FeedForward, self).__init__()
        self.net = tf.keras.Sequential([
            layers.Conv2D(dim, kernel_size=(3, 3), padding='same', activation='relu'),
            layers.BatchNormalization()
        ])

    def call(self, x):
        return self.net(x)


In [20]:
class Former(layers.Layer):
    def __init__(self, dim, num_heads):
        super(Former, self).__init__()
        self.attn = Attention(dim, num_heads)
        self.ff = FeedForward(dim)

    def call(self, x):
        attn_out = self.attn(x)
        ff_out = self.ff(attn_out)
        return ff_out + x


In [21]:
class BaseBlock(layers.Layer):
    def __init__(self, inp=16, out=32, ks=3, stride=2, num_heads=1):
        super(BaseBlock, self).__init__()
        self.former = Former(out, num_heads)
        self.downsample_layer = tf.keras.Sequential([
            layers.Conv2D(inp * 4, kernel_size=ks, strides=stride, padding='same', groups=inp),
            layers.Conv2D(out, kernel_size=1, padding='valid'),
            layers.BatchNormalization()
        ])

    def call(self, x):
        x = self.downsample_layer(x)
        x = self.former(x)
        return x


In [22]:
class HViT(tf.keras.Model):
    def __init__(self, args, cfg, input_shape):
        super(HViT, self).__init__()
        self.patchembed = PatchEmbed()
        self.C2Tblock1 = BaseBlock(**cfg['C2T1'])
        self.C2Tblock2 = BaseBlock(**cfg['C2T2'])

    def call(self, x):
        x = self.patchembed(x)
        x = self.C2Tblock1(x)
        x = self.C2Tblock2(x)
        return x

In [23]:
class Classifier(tf.keras.Model):
    def __init__(self, feature_size=64):
        super(Classifier, self).__init__()
        self.fc = tf.keras.Sequential([
            layers.Flatten(),  # Flatten the input
            layers.Dense(2, activation='softmax')  # Use softmax activation for binary classification
        ])

    def call(self, x):
        return self.fc(x)


In [28]:
from keras.layers import BatchNormalization
class PatchEmbed(layers.Layer):
    def __init__(self):
        super(PatchEmbed, self).__init__()
        self.proj1 = layers.Conv1D(10, kernel_size=1, strides=1, padding="valid", activation='relu')
        self.norm1 = layers.BatchNormalization()
        self.proj2 = layers.Conv1D(10, kernel_size=3, strides=1, padding="same", activation='relu')

    def call(self, x):
        x1 = self.norm1(self.proj1(x))
        print("x1 shape:", x1.shape)  # Print the shape of x1
        x2 = self.proj2(x)
        print("x2 shape (before pooling):", x2.shape)  # Print the shape of x2 before pooling
        x2_reshaped = Reshape((-1, x2.shape[-1]))(x2)
        x2_pooled = layers.GlobalMaxPooling1D()(x2_reshaped)
        print("x2 shape (after pooling):", x2.shape)  # Print the shape of x2 after pooling
        x2 = layers.Flatten()(x2)  # Flatten x2 to match the dimensions with x1
        return tf.concat([x1, x2], axis=-1)



In [25]:
config_my = {
    'name': 'mfmy',
    'kernel': (3, 3),
    'stem': 16,
    'C2T1': {'inp': 16, 'out': 32, 'ks': 3, 'stride': 2, 'num_heads': 8},  # No list wrapping
    'C2T2': {'inp': 32, 'out': 32, 'ks': 3, 'stride': 1, 'num_heads': 8},  # No list wrapping
    'C2T3': {'inp': 32, 'out': 64, 'ks': 3, 'stride': 2, 'num_heads': 8},  # No list wrapping
    'fc2': 2,  # num_classes
}



In [26]:
# Define the patience value
patience = 5
# Create the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience, verbose=True)

In [29]:
epochs = 20
model_lr = 0.001  # Learning rate for the main model
head_lr = 0.0001  # Learning rate for the classifier
weight_decay = 0.01
batch_size = 16
patience = 3
args=""
input_shape = (1802, 12, 129)

for fold, (train_index, test_index) in enumerate(kf.split(all_eeg_augmented)):
    print(f"Fold {fold+1}/{k_folds}")

    # Split data into training and testing sets
    X_train, X_test = all_eeg_augmented[train_index], all_eeg_augmented[test_index]
    y_train, y_test = all_labels[train_index], all_labels[test_index]

    # Create model
    model =  HViT(args, config_my, input_shape)
    classifier = Classifier()

    # Create optimizer
    optimizer = Adam(learning_rate=model_lr)

    # Compile the model with the optimizer
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    early_stopping = EarlyStopping(patience=patience, verbose=True)

    for epoch in range(epochs):
        # Train the model
        history = model.fit(X_train, y_train, batch_size=batch_size, epochs=1, verbose=1)

        # Evaluate the model
        loss, accuracy = model.evaluate(X_test, y_test)

        # Print metrics
        print(f'Epoch {epoch + 1}/{epochs} - Loss: {loss:.6f} - Accuracy: {accuracy:.6f}')

        # Check for early stopping
        if accuracy > early_stopping.best:
            early_stopping.best = accuracy
            model.save_weights('model.h5')
            classifier.save_weights('classifier.h5')
        else:
            patience -= 1

        if patience == 0:
            print("Early stopping...")
            break

    # Load best weights
    model.load_weights('model.h5')
    classifier.load_weights('classifier.h5')
    model.eval()
    classifier.eval()

    # Print shape of data
    print("Shapes of training data and labels:")
    print("X_train shape:", X_train.shape)
    print("y_train shape:", y_train.shape)

    print("Shapes of testing data and labels:")
    print("X_test shape:", X_test.shape)
    print("y_test shape:", y_test.shape)


Fold 1/5
x1 shape: (None, 1802, 12, 10)
x2 shape (before pooling): (None, 1802, 12, 10)


NameError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filefdw8wbtz.py", line 10, in tf__call
        x = ag__.converted_call(ag__.ld(self).patchembed, (ag__.ld(x),), None, fscope)
    File "/tmp/__autograph